# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from holoviews import opts

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ola,59.5833,151.2833,2.86,82,15,1.05,RU,1694805083
1,1,olonkinbyen,70.9221,-8.7187,3.38,74,9,4.65,SJ,1694805083
2,2,port-aux-francais,-49.3500,70.2167,5.59,92,100,20.22,TF,1694805083
3,3,alofi,-19.0595,-169.9187,23.31,70,72,7.33,NU,1694804955
4,4,queenstown,-31.8976,26.8753,17.89,21,100,10.09,ZA,1694805084


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map


map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal_df = city_data_df.loc[
    (city_data_df["Max Temp"] >21) \
    & (city_data_df["Max Temp"] < 27) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
city_ideal_df = city_ideal_df.dropna()

# Display sample data
city_ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,enchanted hills,35.3368,-106.5930,26.33,37,0,1.79,US,1694805089
86,86,jamestown,42.0970,-79.2353,21.68,45,0,2.06,US,1694805025
94,94,reserva,-24.6503,-50.8506,21.56,29,0,1.51,BR,1694805101
98,98,coeur d'alene,47.6777,-116.7805,26.08,37,0,2.57,US,1694805102
109,109,at taj,24.2049,23.2857,26.64,14,0,4.48,LY,1694805104
149,149,colonia,50.9333,6.9500,21.08,69,0,2.57,DE,1694804915
200,200,hamilton,39.1834,-84.5333,25.99,44,0,3.09,US,1694805129
231,231,hancheng,32.5186,112.3522,21.97,80,0,1.87,CN,1694805135
253,253,richards bay,-28.7830,32.0377,22.32,66,0,3.57,ZA,1694805139
270,270,dowlatabad,32.7998,51.6955,22.82,23,0,3.09,IR,1694805144


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,"Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,enchanted hills,US,35.3368,-106.5930,37,
86,jamestown,US,42.0970,-79.2353,45,
94,reserva,BR,-24.6503,-50.8506,29,
98,coeur d'alene,US,47.6777,-116.7805,37,
109,at taj,LY,24.2049,23.2857,14,
149,colonia,DE,50.9333,6.9500,69,
200,hamilton,US,39.1834,-84.5333,44,
231,hancheng,CN,32.5186,112.3522,80,
253,richards bay,ZA,-28.7830,32.0377,66,
270,dowlatabad,IR,32.7998,51.6955,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"


params = {
    "categories":categories,
    "apiKey":geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
enchanted hills - nearest hotel: Santa Ana Star Casino Hotel
jamestown - nearest hotel: DoubleTree Jamestown
reserva - nearest hotel: Hotel Santa Rosa
coeur d'alene - nearest hotel: Roosevelt Inn
at taj - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
hamilton - nearest hotel: North Vista Manor
hancheng - nearest hotel: No hotel found
richards bay - nearest hotel: Bon Hotel Waterfront
dowlatabad - nearest hotel: هتل آفتاب اصفهان
la tuque - nearest hotel: Auberge La Résidence
arish - nearest hotel: سينا
paris - nearest hotel: Hôtel Esmerelda
hwange - nearest hotel: No hotel found
denov - nearest hotel: Turon Hotel
demre - nearest hotel: hotel Kiyak
letlhakane - nearest hotel: Boteti Hotel
chiredzi - nearest hotel: Nesbitt Arms Hotel
huangmei - nearest hotel: No hotel found
abadeh - nearest hotel: No hotel found
mulanje - nearest hotel: No hotel found
tonota - nearest hotel: No hotel found
marion - nearest hotel: Hilton Garden Inn In

,City,Country,Lat,Lng,Humidity,Hotel Name
29,enchanted hills,US,35.3368,-106.5930,37,Santa Ana Star Casino Hotel
86,jamestown,US,42.0970,-79.2353,45,DoubleTree Jamestown
94,reserva,BR,-24.6503,-50.8506,29,Hotel Santa Rosa
98,coeur d'alene,US,47.6777,-116.7805,37,Roosevelt Inn
109,at taj,LY,24.2049,23.2857,14,No hotel found
149,colonia,DE,50.9333,6.9500,69,Wasserturm Hotel Cologne
200,hamilton,US,39.1834,-84.5333,44,North Vista Manor
231,hancheng,CN,32.5186,112.3522,80,No hotel found
253,richards bay,ZA,-28.7830,32.0377,66,Bon Hotel Waterfront
270,dowlatabad,IR,32.7998,51.6955,23,هتل آفتاب اصفهان


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "Hotel Name",
    hover_cols = ["City", "Humidity","Hotel Name", "Country"]
)

# Display the map

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)